In [24]:
from general_tools.in_out.basics import files_in_subdirs, copy_folder_structure
from geo_tool import Point_Cloud, Mesh
from geo_tool.solids import mesh_cleaning as cleaning
import os.path as osp
from multiprocessing import Pool
from tf_lab.data_sets.model_net import pc_sampler

In [25]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# Parameters
n_threads = 50
n_pc_samples = 2048
top_mesh_dir = '/orions4-zfs/projects/optas/DATA/Meshes/Model_Net/Model_Net_40/OBJ_Original/'
top_out_dir = '/orions4-zfs/projects/optas/DATA/Point_Clouds/Model_Net_40/Original/'
top_out_dir = osp.join(top_out_dir, str(n_pc_samples) + osp.sep)
copy_folder_structure(top_mesh_dir, top_out_dir)

In [27]:
ending = '.obj'
search_pattern = ending + '$'
all_in_files = [f for f in files_in_subdirs(top_mesh_dir, search_pattern)]
print len(all_in_files)

12311


In [28]:
drop = len(ending)
pool = Pool(n_threads)
print 'Starting a pool with %d workers.' % (n_threads, )

for in_f in all_in_files:
    out_f = in_f.replace(top_mesh_dir, top_out_dir)
    out_f = out_f[:-drop]
    pool.apply_async(pc_sampler, (in_f, n_pc_samples, out_f))
    
pool.close()
pool.join()

Starting a pool with 50 workers.


/orions4-zfs/projects/optas/Git_Repos/geo_tool/solids/mesh.py:325: RuntimeWarning: invalid value encountered in divide
  face_areas = face_areas / np.sum(face_areas)
/orions4-zfs/projects/optas/Git_Repos/geo_tool/solids/mesh.py:325: RuntimeWarning: invalid value encountered in divide
  face_areas = face_areas / np.sum(face_areas)
/orions4-zfs/projects/optas/Git_Repos/geo_tool/solids/mesh.py:353: RuntimeWarning: overflow encountered in long_scalars
  sample_face_idx[acc: acc + _n_sample] = face_idx
/orions4-zfs/projects/optas/Git_Repos/geo_tool/solids/mesh.py:354: RuntimeWarning: overflow encountered in long_scalars
  acc += _n_sample


In [30]:
all_in_files = [f for f in files_in_subdirs(top_out_dir, '.ply')]
print len(all_in_files)

12310


In [23]:
# Do it specifically for point-clouds that failed to be Manifoldized.
top_in_loc = '/orions4-zfs/projects/optas/DATA/Meshes/Model_Net/Model_Net_40/OBJ_Original/'

top_out_loc = '/orions4-zfs/projects/optas/DATA/Point_Clouds/Model_Net_40/from_manifold/'

manifold_missed = ['door/train/door_0059.obj', 
                   'plant/train/plant_0078.obj',
                   'person/test/person_0093.obj'
                  ]
for n_pc in [2048, 4096]:
    for m_f in manifold_missed:
        mesh_file = osp.join(top_in_loc, m_f)
        out_f = osp.join(top_out_loc, str(n_pc), m_f.replace('.obj',''))
        pc_sampler(mesh_file, n_pc, save_file=out_f)
